In [1]:
import pandas as pd
import numpy as np
import io
import sys
import os.path
import urllib.request
from tqdm import tqdm
from os import listdir
from PIL import Image
import glob
import warnings
warnings.filterwarnings('ignore')
import movieposters as mp

pd.set_option('display.max_colwidth', -1)
np.set_printoptions(threshold=sys.maxsize)

In [2]:
df = pd.read_csv("Data/new.csv")
df.head()

,Title,Release_Year,Genre,Synopsis,Poster_URL,Movie_ID,Movie_URL,Duration,Certification,Voters,Rating
0,Stowaway,(I) (2021),"Adventure,Sci-Fi,Thriller",A three-person crew on a mission to Mars faces an impossible choice when an unplanned passenger jeopardizes the lives of everyone on board.,"https://m.media-amazon.com/images/M/MV5BMWYwMWQxMmUtNjU3MS00NjIwLTllYWMtNzcxYzRkMWYyYTkwXkEyXkFqcGdeQXVyMDM2NDM2MQ@@._V1_UX67_CR0,0,67,98_AL_.jpg",9203694,https://www.imdb.com/title/tt9203694/,116 min,13,"47,561",5.6
1,Anna Karenina,(I) (2012),"Drama,Romance","In late-19th-century Russian high society, St. Petersburg aristocrat Anna Karenina enters into a life-changing affair with the dashing Count Alexei Vronsky.","https://m.media-amazon.com/images/M/MV5BMTU0NDgxNDg0NV5BMl5BanBnXkFtZTcwMjE4MzkwOA@@._V1_UX67_CR0,0,67,98_AL_.jpg",1781769,https://www.imdb.com/title/tt1781769/,129 min,18,"101,744",6.6
2,3rd Rock from the Sun,(1996–2001),"Comedy,Family,Sci-Fi","A group of aliens are sent to Earth, disguised as a human family, to experience and report life on the third planet from the sun.","https://m.media-amazon.com/images/M/MV5BMGMwZWU4ZjAtNTA3MC00ZDFkLWE3YzQtZjdiMTk3ZDJiMDM0XkEyXkFqcGdeQXVyNTA4NzY1MzY@._V1_UY98_CR1,0,67,98_AL_.jpg",115082,https://www.imdb.com/title/tt0115082/,60 min,NaN,"53,126",7.8
3,Toy Story 4,(2019),"Animation,Adventure,Comedy","When a new toy called ""Forky"" joins Woody and the gang, a road trip alongside old and new friends reveals how big the world can be for a toy.","https://m.media-amazon.com/images/M/MV5BMTYzMDM4NzkxOV5BMl5BanBnXkFtZTgwNzM1Mzg2NzM@._V1_UX67_CR0,0,67,98_AL_.jpg",1979376,https://www.imdb.com/title/tt1979376/,100 min,U,"254,387",7.7
4,Archive 81,(2022),"Drama,Horror,Mystery",An archivist hired to restore a collection of tapes finds himself reconstructing the work of a filmmaker and her investigation into a dangerous cult.,"https://m.media-amazon.com/images/M/MV5BZWRkYTU3MzktMjc4Mi00MTI0LWJmYWItYzhmMTRkZTcxMTVkXkEyXkFqcGdeQXVyMTEzMTI1Mjk3._V1_UX67_CR0,0,67,98_AL_.jpg",13365348,https://www.imdb.com/title/tt13365348/,60 min,18,"51,350",7.3


In [3]:
df.shape

(10000, 11)

In [4]:
df.drop_duplicates(subset="Movie_ID", inplace=True)
check_nan_in_df = df.isnull().values.any()
print (check_nan_in_df)

True


In [5]:
#Remove NaN valued entries
df= df.dropna(how='any')
df.shape

(6914, 11)

In [6]:
df

,Title,Release_Year,Genre,Synopsis,Poster_URL,Movie_ID,Movie_URL,Duration,Certification,Voters,Rating
0,Stowaway,(I) (2021),"Adventure,Sci-Fi,Thriller",A three-person crew on a mission to Mars faces an impossible choice when an unplanned passenger jeopardizes the lives of everyone on board.,"https://m.media-amazon.com/images/M/MV5BMWYwMWQxMmUtNjU3MS00NjIwLTllYWMtNzcxYzRkMWYyYTkwXkEyXkFqcGdeQXVyMDM2NDM2MQ@@._V1_UX67_CR0,0,67,98_AL_.jpg",9203694,https://www.imdb.com/title/tt9203694/,116 min,13,"47,561",5.6
1,Anna Karenina,(I) (2012),"Drama,Romance","In late-19th-century Russian high society, St. Petersburg aristocrat Anna Karenina enters into a life-changing affair with the dashing Count Alexei Vronsky.","https://m.media-amazon.com/images/M/MV5BMTU0NDgxNDg0NV5BMl5BanBnXkFtZTcwMjE4MzkwOA@@._V1_UX67_CR0,0,67,98_AL_.jpg",1781769,https://www.imdb.com/title/tt1781769/,129 min,18,"101,744",6.6
3,Toy Story 4,(2019),"Animation,Adventure,Comedy","When a new toy called ""Forky"" joins Woody and the gang, a road trip alongside old and new friends reveals how big the world can be for a toy.","https://m.media-amazon.com/images/M/MV5BMTYzMDM4NzkxOV5BMl5BanBnXkFtZTgwNzM1Mzg2NzM@._V1_UX67_CR0,0,67,98_AL_.jpg",1979376,https://www.imdb.com/title/tt1979376/,100 min,U,"254,387",7.7
4,Archive 81,(2022),"Drama,Horror,Mystery",An archivist hired to restore a collection of tapes finds himself reconstructing the work of a filmmaker and her investigation into a dangerous cult.,"https://m.media-amazon.com/images/M/MV5BZWRkYTU3MzktMjc4Mi00MTI0LWJmYWItYzhmMTRkZTcxMTVkXkEyXkFqcGdeQXVyMTEzMTI1Mjk3._V1_UX67_CR0,0,67,98_AL_.jpg",13365348,https://www.imdb.com/title/tt13365348/,60 min,18,"51,350",7.3
5,Toy Story 3,(2010),"Animation,Adventure,Comedy","The toys are mistakenly delivered to a day-care center instead of the attic right before Andy leaves for college, and it's up to Woody to convince the other toys that they weren't abandoned and to return home.","https://m.media-amazon.com/images/M/MV5BMTgxOTY4Mjc0MF5BMl5BanBnXkFtZTcwNTA4MDQyMw@@._V1_UY98_CR1,0,67,98_AL_.jpg",435761,https://www.imdb.com/title/tt0435761/,103 min,U,"840,449",8.3
...,...,...,...,...,...,...,...,...,...,...,...
9992,Spaced,(1999–2001),"Action,Comedy","Friends Tim and Daisy, 20-something North Londoners with uncertain futures, must pretend to be a couple to live in the only apartment they can afford.","https://m.media-amazon.com/images/M/MV5BMmNjODUxNzctOWU2Zi00ZGU3LWI2MDEtMjZjNjE2MmJiMDZiXkEyXkFqcGdeQXVyNTA4NzY1MzY@._V1_UY98_CR1,0,67,98_AL_.jpg",187664,https://www.imdb.com/title/tt0187664/,25 min,UA 16+,"55,248",8.6
9994,Montana Story,(2021),Drama,"Two estranged siblings return home to the sprawling ranch they once knew and loved, confronting a deep and bitter family legacy against a mythic American backdrop.","https://m.media-amazon.com/images/M/MV5BMTU0ZTBjYTEtNjA5MS00YTM0LWFmMTgtMTgxNWY5NTdjNmZmXkEyXkFqcGdeQXVyMTM1MTE1NDMx._V1_UX67_CR0,0,67,98_AL_.jpg",13656980,https://www.imdb.com/title/tt13656980/,114 min,R,"1,353",6.5
9995,Another 48 Hrs.,(1990),"Action,Comedy,Crime","Jack Cates once again enlists the aid of ex-con Reggie Hammond - this time to take down The Iceman, a ruthless drug lord operating in the San Francisco bay area.","https://m.media-amazon.com/images/M/MV5BNTAzOTE4MzgwNV5BMl5BanBnXkFtZTgwNDgwNTg4NjE@._V1_UX67_CR0,0,67,98_AL_.jpg",99044,https://www.imdb.com/title/tt0099044/,95 min,UA,"42,523",5.9
9996,The Comeback Trail,(2020),"Comedy,Crime",Two movie producers who owe money to the mob set up their aging movie star for an insurance scam to try and save themselves. But they wind up getting more than they ever imagined.,"https://m.media-amazon.com/images/M/MV5BMDg5OTMwMTgtN2NlOS00ZjY5LWI1Y2QtYzEyODgzZDExOGIzXkEyXkFqcGdeQXVyMTkxNjUyNQ@@._V1_UY98_CR1,0,67,98_AL_.jpg",5420210,https://www.imdb.com/title/tt5420210/,104 min,R,"9,654",5.7


In [7]:
#df = df.sample(100)
df['Movie_ID'] = df['Movie_ID'].astype(str)
df = df.sort_values(by = 'Movie_ID')

In [8]:
#Method 1 to download images
not_found = []
for index, row in tqdm(df.iterrows()):
    url = row['Poster_URL']
    imdb_id = row['Movie_ID']
    
    file_path = "Posters/" + str(imdb_id) + ".jpg"
    
    try:
        response = urllib.request.urlopen(url)
        data = response.read()
        file = open(file_path, 'wb')
        file.write(bytearray(data))
        file.close()
    except:
        not_found.append(imdb_id)
        
print("Number of posters not found = ", len(not_found))
print("Following IMDB_ID posters were not found::", not_found)

6914it [23:06,  4.99it/s]

Number of posters not found =  0
Following IMDB_ID posters were not found:: []


In [9]:
# Remove from dataframe those whose posters were not found
df = df[~df['Movie_ID'].isin(not_found)]
df.shape

(6914, 11)

In [10]:
# Check which downloaded images are corrupt and remove them 
bad_images = []
for file in glob.glob("Posters/*.jpg"):

    try:
        img = Image.open(file) # open image file
        img.verify() # verify its an image
    except (IOError, SyntaxError) as e:
        print('Bad file:', file) 

        bad_images.append(file)
        
print("Number of corrupt files:", len(bad_images))

Number of corrupt files: 0


In [11]:
#Checking if all the imdb_id listed here actually have its poster image

image_list = []
for file in glob.glob("Posters/*.jpg"):
    image_list.append(file)
    
print("Number of files found:", len(image_list))
df["Path"] = image_list

Number of files found: 6914


In [12]:
df.to_csv('Data/data_with_path.csv', index=False)
display(df)

,Title,Release_Year,Genre,Synopsis,Poster_URL,Movie_ID,Movie_URL,Duration,Certification,Voters,Rating,Path
617,The Medium,(2021),"Drama,War",An Ex-French Soldier begins participating in underground street fights in order to make money for his brother's family.,"https://m.media-amazon.com/images/M/MV5BNDA4ZDVlOWMtNzAzMi00YmYwLTljYjctYzQ1MzcyOWZmYzdmXkEyXkFqcGdeQXVyMjUzOTY1NTc@._V1_UX67_CR0,0,67,98_AL_.jpg",100029,https://www.imdb.com/title/tt0100029/,108 min,A,"37,253",6.2,Posters\100029.jpg
5923,The Unforgivable,(2021),Comedy,Two couples rent a vacation home for what should be a celebratory weekend get-away.,"https://m.media-amazon.com/images/M/MV5BMjMwOGEyMTQtMDllMC00YTU3LTgyNzktYmEwY2Y4MjQzYmRkXkEyXkFqcGdeQXVyMjM4OTYzMTQ@._V1_UX67_CR0,0,67,98_AL_.jpg",10003008,https://www.imdb.com/title/tt10003008/,52 min,R,"18,464",5.9,Posters\10003008.jpg
6829,Hidden,(I) (2015),"Horror,Sci-Fi,Thriller",Based on the popular diary of the anonymous sex worker known only as Belle de Jour.,"https://m.media-amazon.com/images/M/MV5BMTM1Njg3MzgwOF5BMl5BanBnXkFtZTcwNzMxMzU3MQ@@._V1_UY98_CR5,0,67,98_AL_.jpg",1000734,https://www.imdb.com/title/tt1000734/,43 min,UA,"158,121",7.5,Posters\1000734.jpg
6947,Royal Pains,(2022– ),"Action,Crime,Drama",A New York City writer on sex and love is finally getting married to her Mr. Big. But her three best girlfriends must console her after one of them inadvertently leads Mr. Big to jilt her.,"https://m.media-amazon.com/images/M/MV5BMTYyMzYxMjM3OF5BMl5BanBnXkFtZTcwNjg2OTA3MQ@@._V1_UX67_CR0,0,67,98_AL_.jpg",1000774,https://www.imdb.com/title/tt1000774/,96 min,15+,"92,249",5.7,Posters\1000774.jpg
8863,Blood of Zeus,(2020– ),"Animation,Action,Adventure","A commoner living in ancient Greece, Heron discovers his true heritage as a son of Zeus, and his purpose: to save the world from a demonic army.","https://m.media-amazon.com/images/M/MV5BOWEzZmI2NTItMTFhZi00OWY0LWJiYTktMzIzYzRhNjA1N2IyXkEyXkFqcGdeQXVyMTUzMTM1NDU@._V1_UY98_CR15,0,67,98_AL_.jpg",10009170,https://www.imdb.com/title/tt10009170/,30 min,18,"18,832",7.5,Posters\10009170.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...
6078,Bad Blood,(2017–2018),"Crime,Drama","Mourning his dead child, a haunted Vietnam War veteran attempts to uncover his past while suffering from a severe case of dissociation. To do so, he must decipher reality and life from his own dreams, delusions, and perceptions of death.","https://m.media-amazon.com/images/M/MV5BZTg5MTMyNjktNTZhOC00MGFlLWFlMTMtZGU2MjE3OWQ3NjJkXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_UX67_CR0,0,67,98_AL_.jpg",99871,https://www.imdb.com/title/tt0099871/,113 min,UA,"532,996",7.6,Posters\99871.jpg
2339,The Old Ways,(II) (2020),"Drama,Fantasy,Horror","When a hypochondriac learns that he is dying, he accepts an offer to throw himself in a volcano at a tropical island, and along the way there, learns to truly live.","https://m.media-amazon.com/images/M/MV5BYWViOTc3M2ItZDI5Yy00OGMwLWEyYzMtNGY5NDBhMTkyZDkxXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX67_CR0,0,67,98_AL_.jpg",99892,https://www.imdb.com/title/tt0099892/,23 min,UA,"178,961",6.4,Posters\99892.jpg
6970,After We Collided,(2022– ),Drama,"A tough cop must pose as a kindergarten teacher in order to locate a dangerous criminal's ex-wife, who may hold the key to putting him behind bars.","https://m.media-amazon.com/images/M/MV5BMjMyMTIyOTc0N15BMl5BanBnXkFtZTgwODY1NTk4NjE@._V1_UX67_CR0,0,67,98_AL_.jpg",99938,https://www.imdb.com/title/tt0099938/,111 min,U,"152,480",6.2,Posters\99938.jpg
3696,Against the Ice,(2022),"Comedy,Drama,Romance",A drug kingpin is released from prison and seeks to take total control of the criminal underworld in order to give back to the community.,"https://m.media-amazon.com/images/M/MV5BNTNmMjUzODktOTM5Ni00OTkzLThiYzYtMjgzNzZmN2RlMjZiXkEyXkFqcGdeQXVyMjgyNjk3MzE@._V1_UX67_CR0,0,67,98_AL_.jpg",99939,https://www.imdb.com/title/tt0099939/,124 min,A,"170,115",7.3,Posters\99939.jpg
